### 引用套件

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd

### 宣告常數

In [2]:
domain = 'https://www.tripadvisor.com.tw'
landing_page = 'https://www.tripadvisor.com.tw/Hotels-g293913-Taipei-Hotels.html'
pageurl = 'https://www.tripadvisor.com.tw/Hotels-g293913-oa{}-Taipei-Hotels.html#ACCOM_OVERVIEW' 

### 取得旅館清單

In [3]:
def getHotelListing(url, filter_keywords):
    url_list = []
    res = requests.get(url)
    soup = bs(res.text)
    for hotel in soup.select('.listing'):
        if filter_keywords in hotel.select('.listing_title a')[0].text :
            print hotel.select('.listing_title a')[0].text
            url_list.append(domain + hotel.select('.listing_title a')[0]['href'])
    return url_list

### 取得評論清單

In [4]:
def getPageReview(url, hotelid):
    # 取得頁面資訊
    res = requests.get(url)
    soup = bs(res.text)
    
    # 取得評論ID串
    ary = []
    for review in soup.select('.reviewSelector'):
        ary.append(review['id'].strip('review_'))
        
    # 取得評論查詢URL
    ExpandedURL = 'https://www.tripadvisor.com.tw/ExpandedUserReviews-{}?target={}&context=1&reviews={}&servlet=Hotel_Review&expand=1'
    review_str = ','.join(ary)
    single_review =  review_str.split(',')[0]    
    review_url    =  ExpandedURL.format(hotelid, single_review, review_str)
    
    # 取得評論    
    return getReview(review_url)

### 取得評論

In [5]:
def getReview(review_url):
    review_list = []
    res = requests.get(review_url)
    soup = bs(res.text)  
    for review in soup.select('.extended'):                 
        if len(review.select('.rate > img')) > 0:
            quotes = review.select('.noQuotes')[0].text
            rating = review.select('.rate > img')[0]['alt']
            ratingdate =  review.select('.ratingDate')[0].get('title', review.select('.ratingDate')[0].text)    
            ratingdate = ratingdate.strip().strip('的評論'.decode('utf-8'))
            ratingdate = datetime.strptime(ratingdate.encode('utf-8'), '%Y年%m月%d日')
            comment =review.select('.entry')[0].text            
            mgres = ''
            if len(review.select('.displayText')) > 0:
                mgres = review.select('.entry')[0].text
            review_list.append({'mgrresponse':mgres , 'quotes':quotes, 'rating':rating, 'ratingdate':ratingdate, 'comment':comment})
    return review_list

### 取得旅館列表頁數

In [165]:
res = requests.get(landing_page)
soup = bs(res.text)
hotel_pagenum = int(soup.select('.pageNum')[-1].text)

### 產生每頁連結

In [166]:
pageurls = [landing_page]
for page in range(1,hotel_pagenum):
    pageurls.append(pageurl.format(page * 30))

### 產生旅館連結

In [167]:
total_hotels= []
for url in pageurls:
    total_hotels.extend(getHotelListing(url, '丹迪'.decode('utf-8')))   

丹迪旅店 – 大安森林公園店 
丹迪旅店 – 天母店
丹迪旅店 - 天津店
丹迪旅店 – 大安森林公園店 


In [ ]:
dfs = []
for hotel in total_hotels:
    hottelid = hotel.split('Hotel_Review-')[1].split('-Reviews')[0]
    ExpandedURL = 'https://www.tripadvisor.com.tw/ExpandedUserReviews-{}?target={}&context=1&reviews={}&servlet=Hotel_Review&expand=1'
    df = pd.DataFrame(getPageReview(hotel, hottelid))
    df['hotelurl'] = hotel
    dfs.append(df)

In [169]:
hoteldf = pd.concat(dfs)

In [176]:
import sqlite3 as lite
with lite.connect('tripadvisor2.sqlite') as db:
    hoteldf.to_sql(name='comments', index=False, con=db, if_exists='replace')